# Projekt zaliczeniowy - Badania Internetowe 2022

### *Michał Pawlewski & Wojciech Kabatc*

## Cel badania

Celem naszego badania jest porównanie trzech estymatorów na podstawie oszacowania średnich, obciążeń, wariancji oraz MSE, czyli różnicy między estymatorem a wartością estymowaną.

W badaniu zastosowano następujące metody:



*   Metodę naiwną - obliczenie średniej metodą naiwną.

*   Kalibrację - metoda wymagająca znajomości wartości X
globalnych / średnich z populacji. Pozwala zastosować zmienne jakościowe i ilościowe.

*   Post-stratyfikację - metoda, która jest szczególnym przypadkiem kalibracji. Również wymaga znajomości wartości globalnych / średnich z populacji.

## Opis symulacji

Dane na podstawie, których przeprowadzono badanie zostały pobrane z portalu kaggle.com i zawierają informacje na temat efektywności nauki studentów. Do obliczeń wzięto pod uwagę 8 zmiennych:

* płeć - mężczyźni lub kobiety
* miejsce zamieszkania - zmienna, która przyjmuje wartości wieś (R - rural) lub miasto (U - urban)
* wykształcenie ojca - zmienna wyrażona w postaci rangi od 1 do 5 oznaczająca poziom wykształcenia, im wyższa ranga tym lepsze wykształcenie
*dalc - dzienne spożycie alkoholu wyrażone w postaci rangi od 1 do 5, im wyższa ranga tym osoba spożywa więcej alkoholu dziennie
*czas poświęcany na naukę - zmienna wyrażona w postaci rangi od 1 do 4, im wyższa ranga tym student poświęca więcej czasu na naukę
*czas podróżowania - zmienna wyrażona w postaci rangi od 1 do 4, im wyższa ranga tym student poświęca więcej czasu na podróże
*czas wolny - zmienna wyrażona w postaci rangi od 1 do 5, im wyższa ranga tym student poświęca więcej czasu na czynności wykonywane w wolnym czasie
*nieobecności - ogólna liczba nieobecności

Za zmiennną celu przyjęto liczbę nieobecności na zajęciach, która została oszacowanana za pomocą wcześniej wymienionych metod. Na końcu badania dane oryginalne zestawiono z danymi oszacowanymi.

Każdy estymator jest liczony dla 100 prób nielosowych i następnie uśredniona wartość zwracana jest do tabeli porównującej metody. Każda z prób nielosowych liczyła 20% wielkości całej populacji. Przyjęto wagę - im wyższa wartość absencji dla ucznia, tym wyższa waga. Na podstawie tego samego zbioru wyników liczone są obciążenia, wariancje i MSE.

Pobranie i wczytanie potrzebnych do badania pakietów.

In [3]:
install.packages("tidyverse")
install.packages("survey")

library(tidyverse)
library(survey)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Sekcja kodu, w której wczytywany jest plik z danymi oraz wybierane są z niego odpowiednie kolumny.



In [5]:
data <- read_csv("student_data.csv")
data <- data %>% select(sex, address, Fedu, Dalc, studytime, traveltime, absences, freetime) %>%
  drop_na()
data <- data %>%
  mutate(
    Fedu = as.character(Fedu),
    Dalc = as.character(Dalc),
    studytime = as.character(studytime),
    traveltime = as.character(traveltime),
    freetime = as.character(freetime)
  )
head(data)


Rows: 395 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (17): school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardi...
dbl (16): age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sex,address,Fedu,Dalc,studytime,traveltime,absences,freetime
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
F,U,4,1,2,2,6,3
F,U,1,1,2,1,4,3
F,U,1,2,2,1,10,3
F,U,2,1,3,1,2,2
F,U,3,1,2,1,4,3
M,U,3,1,2,1,10,4


Metody naiwne

Wylosowanie 100 prób nielosowych z zbioru danych liczącego 20% ilości rekordów w bazie i obliczenie z nich średnich naiwnych, odsetka osób z absencją powyżej 10 oraz mediany naiwnej.

In [6]:
C <- 100
total_rows <- nrow(data)
size <- round(0.2*total_rows)
mean_naive <- numeric(C)
percentage_naive <- numeric(C)
median_naive <- numeric(C)

for (c in 1:C){
  #losowanie próby nielosowej
  id_nielosowa <- sample(x = 1:total_rows, size,
                      prob = data$absences/sum(data$absences))
  proba_nielosowa <- data[id_nielosowa, ]
  #średnia
  mean_naive[c] <- mean(proba_nielosowa$absences)
  #współczynnik uczniów o liczbie nieobecności większej od 10
  percentage_naive[c] <- sum(proba_nielosowa$absences>10)/nrow(proba_nielosowa)
  #mediana
  median_naive[c] <- median(proba_nielosowa$absences)
}

# Kalibracja

Wartości globalne odtwarzane według zmiennych `freetime`, `address`, `studytime` oraz `sex` z wykorzystaniem funckji `xtabs`. Tworzony jest obiekt `survey.design` na podstawie próby nielosowej. Sama kalibracja odbywa się dzięki funkcji `calibrate`. Przeprowadzane są dwie kalibracje z oraz bez zmiennej `address`. Na samym końcu wyniki, czyli średnie, są przypisywane do wektorów `mean_kalibracja` oraz `mean_kalibracja2`.

UWAGA!: Wyświetlenie komunikatów `Warning message in svydesign.default(ids = ~1, data = proba_nielosowa): “No weights or probabilities supplied, assuming equal probability”` nie świadczy o błędzie w kodzie, ale o pominięciu przypisania wag w tej metodzie estymacji, co jest zgodne z założeniami naszego badania.


In [7]:
#https://github.com/DepartmentOfStatisticsPUE/bi-2022/blob/main/notebooks/2-quasi-cal.ipynb
# kalibracja dla 100 prób nielosowych
A <- 100
mean_kalibracja <- numeric(A)
mean_kalibracja2 <- numeric(A)

for (a in 1:A){

proba_losowa <- data[sample(nrow(data), size), ]

id_nielosowa <- sample(x = 1:total_rows, size,
                      prob = data$absences/sum(data$absences))
  proba_nielosowa <- data[id_nielosowa, ]

total_freetime <- xtabs( ~ freetime, data = proba_losowa)
total_address <- xtabs( ~ address, data = proba_losowa)
total_studytime <- xtabs( ~ studytime, data = proba_losowa)
total_sex <- xtabs( ~ sex, data = proba_losowa)

svy_nielosowa <- svydesign(ids = ~1,  data = proba_nielosowa)

## pierwszy zestaw
svy_nielosowa_calib <- calibrate(
  design = svy_nielosowa,
  formula = list(~sex, ~studytime),
  population = list(total_sex, total_studytime)
)

## drugi zestaw
svy_nielosowa_calib2 <- calibrate(
  design = svy_nielosowa,
  formula = list(~sex, ~studytime, ~address),
  population = list(total_sex, total_studytime, total_address)
)

mean_kalibracja[a] <- svymean(~absences, svy_nielosowa_calib)[1]
mean_kalibracja2[a] <- svymean(~absences, svy_nielosowa_calib2)[1]
}

Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesign.default(ids = ~1, data = proba_nielosowa):
“No weights or probabilities supplied, assuming equal probability”
Warning message in svydesig

In [8]:
svy_nielosowa_calib

Independent Sampling design (with replacement)
calibrate(design = svy_nielosowa, formula = list(~sex, ~studytime), 
    population = list(total_sex, total_studytime))

# Post-stratyfikacja


In [9]:


B <- 100
wyniki_naiwny <- numeric(B)
wyniki_ps <- numeric(B)
mean_ps <- numeric(B)

## wartości globalne
pop_sex_address <- xtabs(~ sex + address, data = data)

for (b in 1:B) {
  id_nielosowa <- sample(x = 1:total_rows, size = 0.2*total_rows, prob = data$absences/sum(data$absences))
  proba_nielosowa <- data[id_nielosowa, ]
  ## wyliczam estymator naiwny
  wyniki_naiwny[b] <- mean(proba_nielosowa$absences)
  ## przygotowanie do post-stratyfikcji
  svydes <- svydesign(ids = ~1, data = proba_nielosowa, weights = ~1)
  ## post-stratyfikacja
  svydes_post <- postStratify(svydes, ~ sex + address, pop_sex_address)
  ## wyliczam estymator post-stratyfikacyjny
  mean_ps[b] <- svymean(~ absences, svydes_post)[1]
}

mean_ps

[1] 11.74482 12.48245 13.58500 13.62773 12.96262 13.20387 12.40510 12.85675
  [9] 13.79005 12.92903 12.28568 12.46162 13.65675 13.05514 12.99748 13.29652
 [17] 13.90385 14.40278 13.50817 13.26902 14.12508 13.08020 13.57271 13.63745
 [25] 13.43496 12.48132 13.15524 13.04037 13.17839 13.91797 13.70101 12.94307
 [33] 13.71445 13.89406 13.70656 13.73632 13.55981 14.02431 13.68108 14.28646
 [41] 12.90285 12.99729 12.83055 13.12412 13.67846 12.55878 13.49357 12.74518
 [49] 12.78144 12.95486 12.09943 13.18569 13.76821 12.39713 13.77081 14.88625
 [57] 13.11891 13.80250 12.83307 13.50579 13.27359 13.81652 13.11814 13.12397
 [65] 12.42896 13.56589 13.91663 13.34303 13.39775 14.58512 13.30729 14.27619
 [73] 13.31656 12.63150 13.57081 12.15442 12.09970 12.90675 12.82149 12.58674
 [81] 12.33385 11.59362 13.33690 12.62619 14.58858 13.10107 12.26563 13.86005
 [89] 13.30073 13.44644 13.09451 12.80181 13.17304 12.59310 13.83102 12.76786
 [97] 13.24345 14.56504 13.34282 13.15523

# Wyniki

Porównujemy wszystkie wyniki w tabelce.

In [10]:
wyniki <- data.frame(
  populacja = mean(data$absences),
  naive = mean(mean_naive),
  kalibracja = mean(mean_kalibracja),
  kalibracja2 = mean(mean_kalibracja2),
  ps = mean(mean_ps)
)
rownames(wyniki) <- "srednia"

wyniki_obc <- wyniki[1,] - mean(data$absences)
rownames(wyniki_obc) <- "obciazenie"

wyniki_var <- data.frame(
  populacja = 0,
  naive = var(mean_naive),
  kalibracja = var(mean_kalibracja),
  kalibracja2 = var(mean_kalibracja2),
  ps = var(mean_ps))
rownames(wyniki_var) <- "wariancja"

wyniki_MSE <- data.frame(
  populacja = wyniki_obc[1,1]^2+wyniki_var[1,1],
  naive = wyniki_obc[1,2]^2+wyniki_var[1,2],
  kalibracja = wyniki_obc[1,3]^2+wyniki_var[1,3],
  kalibracja2 = wyniki_obc[1,4]^2+wyniki_var[1,4],
  ps = wyniki_obc[1,5]^2+wyniki_var[1,5])
rownames(wyniki_MSE) <- "MSE"

wyniki <- rbind(wyniki, wyniki_obc, wyniki_var, wyniki_MSE)
wyniki

,populacja,naive,kalibracja,kalibracja2,ps
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
srednia,5.708861,13.2805063,12.8131782,12.8300164,13.2399214
obciazenie,0.000000,7.5716456,7.1043174,7.1211557,7.5310606
wariancja,0.000000,0.3595892,0.3969169,0.4190974,0.3984198
MSE,0.000000,57.6894059,50.8682431,51.1299553,57.1152936


# Podsumowanie

Dla wszystkich estymatorów uzyskano wysokie obciążęnia. To oznacza, że odbiegają one istotnie od wartości tych wskaźników dla całej populacji. Dla metody naiwnej oraz post-stratyfikacji obciążenia są zbliżone. Nieco inaczej wypadają estymatory kalibracyjne. Oszacowane liczby nieobecności mocno odbiegają od wartości oryginalnych.

Wyniki najbardziej zróżnicowane, czyli te o największej wariancji uzyskane zostały w metodzie kalibracji. W przypadku średnich prezentowały się one podobnie wśród wszystkich metod.

Największy błąd średniokwadratowy wskazujący na duże różnice między wartością oszacowaną, a oryginalną wykazał estymator naiwny. Post-stratyfikacja posiada błąd na podobnym poziomie. Nieco lepiej wypadł estymator kalibracyjny, który jednak posiada znaczący błąd MSE.

Różnice pomiędzy wartościami dla estymatorów, a dla całej populacji są spowodowane przyjęciem wagi przy losowaniu próby nielosowej. Uczniowie, którzy mieli większą liczbę nieobecności uzyskali większą wagę w losowaniu, co przełożyło się na widoczne powyżej różnice.